In [1]:
import pandas as pd
import plotly.express as px

In [2]:
raw_df = pd.read_csv("./files/flights.csv")

C:\Users\Diego Dos Santos\AppData\Local\Temp\ipykernel_45012\117606716.py:1: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_df = pd.read_csv("./files/flights.csv")


In [3]:
# Remove rows with missing ARRIVAL_DELAY values
df = raw_df[raw_df.ARRIVAL_DELAY.isna() == False]

In [4]:
# Get 10k random samples
df = df.sample(n=10000, random_state=42)
print(df.shape)

(10000, 31)


In [5]:
fig = px.histogram(df, x=df.MONTH, y="ARRIVAL_DELAY")
fig.show();